Plan is to comapre trotterized dynamics for GKP-TMS/GKP-repetition code vs just bare oscillator under additive gaussian noise (after each trotter step). We will use the analytical results from [here](https://arxiv.org/abs/1903.12615).

In [12]:
import sys
import os

# Adjust the path based on your directory structure
module_path = os.path.abspath(os.path.join('..', '..', '..'))  # Moves three directories up
if module_path not in sys.path:
    sys.path.append(module_path)
    
# Now you can import c2qa and other modules from bosonic-qiskit
import c2qa
import qiskit
import numpy as np
import c2qa.util as util
import matplotlib.pyplot as plt
import matplotlib
from dynamics import hopping_term, e_field

In [13]:
# Parameters
numberofmodes = 3
numberofqubits = numberofmodes - 1
numberofqubitspermode = 3
cutoff = 2 ** numberofqubitspermode

# Create registers
qmr = c2qa.QumodeRegister(num_qumodes=numberofmodes, num_qubits_per_qumode=numberofqubitspermode)  # Qumodes
qbr = qiskit.QuantumRegister(size=numberofqubits)  # Qubits for gauge fields
circuit = c2qa.CVCircuit(qmr, qbr)  # Circuit

# Initialize bosonic modes (Fock state |010⟩)
fock_states = [0, 1, 0]  # Corresponds to |010⟩
for i, fock_amplitude in enumerate(fock_states):
    coeffs = [0] * cutoff  # Initialize all coefficients to 0
    coeffs[fock_amplitude] = 1  # Set the amplitude for the desired Fock state
    circuit.cv_initialize(coeffs, qmr[i])  # Initialize each qumode

# Initialize qubits (|+-⟩)
plus_state = [1 / np.sqrt(2), 1 / np.sqrt(2)]  # Coefficients for |+⟩
minus_state = [1 / np.sqrt(2), -1 / np.sqrt(2)]  # Coefficients for |−⟩

# Apply initialization to each qubit
for i in range(numberofqubits):
    if i < 1:  # First qubit to |+⟩
        circuit.initialize(plus_state, qbr[i])
    else:  # Last qubit to |−⟩
        circuit.initialize(minus_state, qbr[i])


Using a two-mode GKP two-mode squeezing (TMS) code, the logical noise as a function of physical noise is given by:

$$
\sigma^*_L \xrightarrow{\sigma \ll 1} \frac{2\sigma^2}{\sqrt{\pi}} \sqrt{\log_e{\left[\frac{\pi^{3/2}}{2\sigma^4}\right]}}.
$$


where $\sigma$ is the standard deviation of the physical quadrature noise.



In [14]:

def agn_channel(circuit, qmr, mode, noise_strength):
    """
    Applies an Additive Gaussian Noise (AGN) channel to a bosonic mode

    Parameters:
        circuit (QuantumCircuit): The quantum circuit to modify.
        qmr (QumodeRegister): The quantum register containing the bosonic modes.
        mode (int): The index of the bosonic mode.
        noise_strength (float): The standard deviation of the Gaussian noise.

    Returns:
        None
    """
    # Generate a random displacement from a Gaussian distribution
    displacement = np.random.normal(0, noise_strength)
    # Apply the displacement to the specified mode
    circuit.cv_d(displacement, qmr[mode])

def calculate_logical_noise(physical_noise):
    """
    Calculates the logical noise given the physical noise

    Parameters:
        physical_noise (float): Standard deviation of the physical quadrature noise.

    Returns:
        float: Standard deviation of the logical quadrature noise.
    """
    import numpy as np

    try:
        # Calculate the logical noise using the equation
        logical_noise = (2 * physical_noise**2 / np.sqrt(np.pi)) * np.sqrt(
            np.log(np.pi**(3/2) / (2 * physical_noise**4))
        )
    except ValueError:
        raise ValueError("Ensure that the physical noise is within a valid range for this formula.")
    
    return logical_noise

In [15]:
def agn_trotter_z2_higgs(numberofmodes, N, g, j, delta_t, noise_strength, noise_type="physical"):
    """
    Trotterizes the \( \mathbb{Z}_2 \)-Higgs model over N steps, with optional logical or physical noise application.

    Parameters:
        numberofmodes (int): The number of bosonic modes.
        N (int): The number of Trotter steps.
        g (float): Coupling constant for the electric field term.
        j (float): Coupling constant for the hopping term.
        delta_t (float): Trotter time step.
        noise_strength (float): Standard deviation of the physical quadrature noise.
        noise_type (str): Type of noise to apply, either "physical" or "logical".

    Returns:
        list: A list of occupation numbers for bosonic modes at each Trotter step.
    """
    # Derived parameters
    numberofqubits = numberofmodes - 1
    numberofqubitspermode = 3
    cutoff = 2**numberofqubitspermode

    # Create registers
    qmr = c2qa.QumodeRegister(num_qumodes=numberofmodes, num_qubits_per_qumode=numberofqubitspermode)  # Qumodes
    qbr = qiskit.QuantumRegister(size=numberofqubits)  # Qubits for gauge fields
    circuit = c2qa.CVCircuit(qmr, qbr)  # Circuit

    # Initialize bosonic modes (Fock state |010⟩)
    fock_states = [0, 1, 0]  # Corresponds to |010⟩
    for i, fock_amplitude in enumerate(fock_states):
        coeffs = [0] * cutoff  # Initialize all coefficients to 0
        coeffs[fock_amplitude] = 1  # Set the amplitude for the desired Fock state
        circuit.cv_initialize(coeffs, qmr[i])  # Initialize each qumode

    # Initialize qubits (|+-⟩)
    plus_state = [1 / np.sqrt(2), 1 / np.sqrt(2)]  # Coefficients for |+⟩
    minus_state = [1 / np.sqrt(2), -1 / np.sqrt(2)]  # Coefficients for |−⟩
    for i in range(numberofqubits):
        if i < 1:  # First qubit to |+⟩
            circuit.initialize(plus_state, qbr[i])
        else:  # Last qubit to |−⟩
            circuit.initialize(minus_state, qbr[i])

    # Begin Trotterization
    occupations = []
    for step in range(N):
        # Apply electric field term to all qubits
        for qb_index in range(numberofqubits):
            e_field(circuit, qbr, qb_index, g, delta_t)

        # Apply hopping term between adjacent modes
        for mode in range(0, numberofmodes - 1):
            hopping_term(circuit, qbr, mode, qmr, mode, mode + 1, j, delta_t)

        # Determine noise strength based on noise type
        if noise_type == "logical":
            current_noise_strength = calculate_logical_noise(noise_strength)
        elif noise_type == "physical":
            current_noise_strength = noise_strength
        else:
            raise ValueError("Invalid noise_type. Choose either 'physical' or 'logical'.")

        # Apply AGN channel to all modes
        for mode in range(numberofmodes):
            agn_channel(circuit, qmr, mode, current_noise_strength)

        # Simulate and collect occupations
        stateop, _, _ = c2qa.util.simulate(circuit)
        occupation = c2qa.util.stateread(stateop, numberofqubits, numberofmodes, cutoff, verbose=False)
        occupations.append(np.array(list(occupation[0][0])))

    return occupations


In [16]:
# Define parameters
n_samples = 50  # Number of Monte Carlo samples
dt = 0.1
N = 3
J = 1
g = 0
numberofmodes = 3
noise_strength = 0.05  # variance

# Arrays to store accumulated results
occupations_physical_avg = np.zeros((N, numberofmodes))
occupations_logical_avg = np.zeros((N, numberofmodes))

for _ in range(n_samples):
    # Run simulations
    occupations_physical = agn_trotter_z2_higgs(
        numberofmodes,
        N,
        g,
        J,
        dt,
        noise_strength,
        noise_type="physical"
    )
    occupations_logical = agn_trotter_z2_higgs(
        numberofmodes,
        N,
        g,
        J,
        dt,
        noise_strength,
        noise_type="logical"
    )

    # Accumulate results
    occupations_physical_avg += np.array(occupations_physical)
    occupations_logical_avg += np.array(occupations_logical)

# Compute averages
occupations_physical_avg /= n_samples
occupations_logical_avg /= n_samples

In [17]:
# Extend occupations arrays to match pcolormesh grid requirements
occupations_array_physical = np.vstack([occupations_physical_avg, np.zeros((1, numberofmodes))])[:-1, :]
occupations_array_logical = np.vstack([occupations_logical_avg, np.zeros((1, numberofmodes))])[:-1, :]

# Define x-axis (modes) and y-axis (time steps)
x_values = np.arange(numberofmodes + 1) - numberofmodes // 2 - 0.5
y_values = np.arange(N + 1) * dt * J

# Ensure the plots directory exists
os.makedirs("plots", exist_ok=True)

# Plot for physical noise
plt.pcolormesh(
    x_values,
    y_values,
    occupations_array_physical,
    cmap='viridis',
    shading='auto',
    vmin=0,
    vmax=np.max(occupations_array_physical)
)
plt.title(f"Physical Noise ($g/J = {int(g / J)}$)")
plt.xlabel("Site")
plt.ylabel("Time ($Jt$)")

cbar = plt.colorbar()
cbar.ax.get_yaxis().labelpad = 15
cbar.set_label("Occupation", rotation=270)

plt.rcParams["figure.figsize"] = (6, 3)
plt.tight_layout()
plt.savefig(os.path.join("plots", "Z2_physical_noise.pdf"), format='pdf', bbox_inches='tight')
plt.close()

# Plot for logical noise
plt.pcolormesh(
    x_values,
    y_values,
    occupations_array_logical,
    cmap='viridis',
    shading='auto',
    vmin=0,
    vmax=np.max(occupations_array_logical)
)
plt.title(f"Logical Noise ($g/J = {int(g / J)}$)")
plt.xlabel("Site")
plt.ylabel("Time ($Jt$)")

cbar = plt.colorbar()
cbar.ax.get_yaxis().labelpad = 15
cbar.set_label("Occupation", rotation=270)

plt.rcParams["figure.figsize"] = (6, 3)
plt.tight_layout()
plt.savefig(os.path.join("plots", "Z2_logical_noise.pdf"), format='pdf', bbox_inches='tight')
plt.close()

print("Plots for physical and logical noise saved in the 'plots' directory.")

Plots for physical and logical noise saved in the 'plots' directory.
